#### Import necessary libraries

In [20]:
import os
import pandas as pd
import nltk

# change directory if necessary
print(os.getcwd())
os.chdir(r"/Users/hokongzheng/Downloads/FIT3162---Zhiwei-and-Friends/") 

/Users/hokongzheng/Downloads/FIT3162---Zhiwei-and-Friends


#### Read dataset

In [22]:
def readfile(filename):
    data = pd.read_excel(filename)
    data = data.dropna(subset=["text"])
    return data

In [23]:
#SemEval dataset
filename = "Restaurants_Train.xlsx"
data = readfile(filename)

#### Preliminary analysis

In [126]:
data.head(5)

,sentences,sentence,id,text,aspectTerms,aspectTerm,term,polarity,from,to,aspectCategories,aspectCategory,category,polarity.1
0,NaN,NaN,3121.0,But the staff was so horrible to us.,NaN,NaN,staff,negative,8.0,13.0,NaN,NaN,service,negative
1,NaN,NaN,2777.0,"To be completely fair, the only redeeming fact...",NaN,NaN,food,positive,57.0,61.0,NaN,NaN,food,positive
3,NaN,NaN,1634.0,"The food is uniformly exceptional, with a very...",NaN,NaN,food,positive,4.0,8.0,NaN,NaN,food,positive
6,NaN,NaN,2534.0,Where Gabriela personaly greets you and recomm...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,service,positive
7,NaN,NaN,583.0,"For those that go once and don't enjoy it, all...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,positive


In [127]:
data.tail(5)

,sentences,sentence,id,text,aspectTerms,aspectTerm,term,polarity,from,to,aspectCategories,aspectCategory,category,polarity.1
4900,NaN,NaN,1063.0,But that is highly forgivable.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,positive
4901,NaN,NaN,777.0,"From the appetizers we ate, the dim sum and ot...",NaN,NaN,appetizers,positive,9.0,19.0,NaN,NaN,food,positive
4905,NaN,NaN,875.0,"When we arrived at 6:00 PM, the restaurant was...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,neutral
4906,NaN,NaN,671.0,Each table has a pot of boiling water sunken i...,NaN,NaN,table,neutral,5.0,10.0,NaN,NaN,food,neutral
4912,NaN,NaN,617.0,I am going to the mid town location next.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,anecdotes/miscellaneous,neutral


In [146]:
keys = pd.DataFrame(data.keys())
keys.index += 1
keys = keys.rename(columns={0:"Keys"})
keys

,Keys
1,sentences
2,sentence
3,id
4,text
5,aspectTerms
6,aspectTerm
7,term
8,polarity
9,from
10,to


In [129]:
reviews = data["text"]
print("Number of Reviews:", len(reviews))

Number of Reviews: 3044


In [152]:
ex_asp = pd.unique(data["category"])
print("Explicit Aspects:", ", ".join(ex_asp))

Explicit Aspects: service, food, anecdotes/miscellaneous, ambience, price


In [153]:
ex_asp_freq_table = data.groupby("category").size().reset_index()
ex_asp_freq_table = ex_asp_freq_table.rename(columns={"category":"Ex.Asp", 0:"Num"})
ex_asp_freq_table.index +=1
ex_asp_freq_table

,Ex.Asp,Num
1,ambience,261
2,anecdotes/miscellaneous,1082
3,food,1158
4,price,133
5,service,410


In [18]:
asp_polarity_freq_table = data.groupby(["category", "polarity.1"]).size().reset_index()
asp_polarity_freq_table = asp_polarity_freq_table.rename(columns={"category":"Ex.Asp", "polarity.1":"Polarity", 0:"Num"})
asp_polarity_freq_table.index +=1
asp_polarity_freq_table

NameError: name 'data' is not defined

# -----------
# Testing 123
# -----------
#### Loading necessary libraries

In [146]:
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D
from keras.layers.core import Reshape, Flatten, Activation
from keras.callbacks import EarlyStopping
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.models import Model
from keras import regularizers

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

#### Preprocess the data

In [70]:
#data['text'] = data['text'].str.lower()

#### Split data into training, testing, validation sets

In [165]:
reviews = data['text']
labels = data['term']
### rmb to remove conflict
x_train, x_test, y_train, y_test = train_test_split(reviews, labels, test_size = 0.4, random_state=999)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size = 0.5, random_state=999)

In [166]:
labels

0                                     staff
1                                      food
3                                      food
6                                       NaN
7                                       NaN
8                                      food
10                                      NaN
12      orrechiete with sausage and chicken
16                                   Bagels
17                                     food
18                                    songs
19                                    toast
26                                   drinks
28                                   design
30                                  cuisine
31                                      NaN
32                                    pizza
34                                      NaN
35                                      NaN
36                                      NaN
37                                      NaN
38                                      NaN
39                              

#### Tokenize text of the training data with keras preprocessing text function

In [72]:
# Set maximum number of words to be embedded
NUM_WORDS = 100000

# Set maximum length of a sentence
MAX_LEN = 65

# Define/Load Tokenize text function
tokenizer = Tokenizer(num_words=NUM_WORD)

# Fit the function on the text
tokenizer.fit_on_texts(x_train)

# Count number of unique tokens
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 3471 unique tokens.


In [73]:
# Convert train and val to sequence
sequences_train = tokenizer.texts_to_sequences(x_train)
sequences_valid= tokenizer.texts_to_sequences(x_val)

In [155]:
# Limit size of train/val to 50 and pad the sequence
x_train = pad_sequences(sequences_train,maxlen=MAX_LEN)
x_val = pad_sequences(sequences_valid,maxlen=x_train.shape[1])

# Convert target to array
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)

# Printing shape
print('Shape of X train and X validation tensor:', x_train.shape,x_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)

Shape of X train and X validation tensor: (1826, 65) (609, 65)
Shape of label train and validation tensor: (1, 1826) (1, 609)


## Word Embedding
#### Using pretrained Word2Vec model from Google or Amazon
#### Create 300-dimensional vectors

In [156]:
#word_vectors = KeyedVectors.load_word2vec_format('./AmazonWE/sentic2vec.csv')
word_vectors = pd.read_csv('./AmazonWE/sentic2vec.csv', encoding = "ISO-8859-1", header=None)

In [157]:
word_vectors.head(5)

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,nycsamsung,0.017007,-0.010965,0.034439,-0.003403,0.001669,-0.016725,-0.009386,-0.014179,0.006383,...,-0.005171,0.027319,-0.007168,0.010212,-0.017336,0.003275,-0.001003,0.011749,-0.004390,0.012654
1,localizer,0.038885,0.059859,-0.051594,0.021452,0.001800,-0.010394,-0.010728,0.096014,0.012083,...,-0.087075,-0.005560,-0.016801,0.044161,-0.010343,0.055683,-0.026794,-0.046406,-0.002826,0.046562
2,gah,0.000715,-0.001516,-0.000800,-0.000104,-0.000850,-0.000913,0.000452,-0.001184,-0.000177,...,-0.000289,0.000471,0.001467,-0.000577,0.000112,-0.000166,0.001002,0.000662,-0.000665,-0.001045
3,hasselfree,0.014361,-0.000866,-0.035235,0.024500,0.034341,-0.010994,-0.012281,0.020808,-0.034909,...,-0.010619,0.013001,-0.021808,0.006658,-0.023384,0.019640,0.028458,-0.030321,-0.042621,-0.011562
4,prendeel,0.025254,0.015678,-0.035391,-0.012458,0.045409,-0.019798,-0.004205,0.014728,-0.012590,...,-0.015791,0.002601,-0.038128,0.007929,0.001784,0.023809,-0.023151,-0.005321,-0.017844,0.033579


In [158]:
EMBEDDING_DIM=300
vocabulary_size=min(len(word_index)+1,(NUM_WORDS))

embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i < vocabulary_size:
        embedding_vector = word_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
#         else:
#             vec = np.zeros(EMBEDDING_DIM)
#             embedding_matrix[i] = vec

In [159]:
del(word_vectors)

In [160]:
# Define embedding function using the embedding_matrix
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True,
                            input_length=MAX_LEN)

In [161]:
del(embedding_matrix)

#### Build CNN with EarlyStopping
What is unknown so far?
- the structure of fully-connected layer
- batch size

In [162]:
sequence_length = x_train.shape[1]
filter_sizes = [2,3]
num_filters = [100, 50]

# Create model
model = Sequential()

# Add layers
model.add(embedding_layer)
model.add(Reshape((sequence_length,EMBEDDING_DIM,1)))
model.add(Conv2D(num_filters[0],(filter_sizes[0], filter_sizes[0]), activation='tanh'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(num_filters[1],(filter_sizes[1], filter_sizes[1]), activation='tanh'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(MAX_LEN, activation='softmax', kernel_regularizer=regularizers.l2(0.01)))
# model.add(Activation('softmax'))

In [163]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# summarize the model
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 65, 300)           1041600   
_________________________________________________________________
reshape_23 (Reshape)         (None, 65, 300, 1)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 64, 299, 100)      500       
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 32, 149, 100)      0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 30, 147, 50)       45050     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 15, 73, 50)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 54750)             0         
__________

In [164]:
history = model.fit(x_train, y_train,
                    epochs=30,
                    batch_size=60,
                    validation_data=(x_val, y_val))

ValueError: Error when checking target: expected dense_24 to have shape (65,) but got array with shape (1826,)